In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load the datasets
num_blocks = 70  # load 70 blocks. Each block has 50 customers

df_all = [
    pd.read_csv("./raw/London_halfhourly_dataset/block_{}.csv".format(i)) for i in range(num_blocks)
]

# pivot each AMI file from the format:
#             LCLid tstp energy(kWh/hh)
# to the format:
#             tstp, ID1, ID2, ..., IDN,
for i in range(num_blocks):
    df_all[i] = df_all[i].pivot(index=['tstp'], columns=['LCLid'], values='energy(kWh/hh)')
    # change datetime format
    df_all[i].index = df_all[i].index.astype('datetime64[ns]')

# create a equally-spaced timestamp series
timestamps = pd.date_range(start='2012-08-01 00:00:00', end='2014-02-28 00:00:00', freq='30min')

In [ ]:
# join all blocks into one dataframe

df = pd.DataFrame(index=timestamps)
for d in df_all:
    # join (join on the index by default)
    df = df.join(d, how='left')
print(df.shape)

In [ ]:
# visualize nan

plt.imshow(df.to_numpy(dtype=np.float32), aspect='auto')
print(df.shape)

In [ ]:
# drop customers with too many nans

# hint: dropna(thresh=k, axis='columns'): keep columns with at least k non-nan values 
df = df.dropna(thresh=int(df.shape[0]*0.9), axis='columns')

plt.figure(figsize=(12 *10, 8 *10))
plt.imshow(df.to_numpy(dtype=np.float32), aspect='auto')
print(df.shape)
print('percentage of NaN: {} * 100%'.format(df.isna().sum().sum()/(df.shape[0] * df.shape[1]) * 100.))

In [ ]:
# matrix factorization to fill the remaining nans
# since the full matrix takes so long, we divide it into a few chunks and process each chunk individually.

from fancyimpute import MatrixFactorization
chunks = 5
size_each_chunk = df.shape[0] // chunks
dfp = []
for i in range(chunks):
    dfp1 = MatrixFactorization(rank=10, epochs=200, 
                learning_rate=0.01).fit_transform(df.iloc[i*size_each_chunk:(i+1)*size_each_chunk, :])
    dfp.append(dfp1)
dfp1 = MatrixFactorization(rank=10, epochs=200, 
                learning_rate=0.01).fit_transform(df.iloc[chunks*size_each_chunk:, :])
dfp.append(dfp1)
dfp = np.vstack(dfp)


In [ ]:
# visualize nan (should be no nan at this point)

plt.figure(figsize=(12 *10, 8 *10))
plt.imshow(dfp, aspect='auto')
print(dfp.shape)

In [ ]:
# aggregate every 5 meters

df = pd.DataFrame(dfp)
df_agg = df.astype(float).groupby(np.arange(len(df.columns))//5, axis=1).mean()


In [ ]:
# save

df_agg.to_csv('./processed/first_{}_ami_aggto_{}.csv'.format(df.shape[1], df_agg.shape[1]), header=False, index=False)
df.to_csv('./processed/first_{}_ami.csv'.format(df.shape[1]), header=False, index=False)